In [3]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-44027?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)


Apply for Sr. Software Engineer - Tech OPS

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySr. Software Engineer - Tech OPSShanghai, China MainlandBecome part of the Converse 

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Sr. Software Engineer - Tech OPS',
 'experience': '7+ years of project management experience and 2+ years of agile project management experience',
 'skills': ['PMP Certificate',
  'Microsoft Office (Word, Excel, PowerPoint, Visio, Project)',
  'Jira',
  'Confluence',
  'Smartsheet',
  'Fluent in both written and verbal Chinese and English'],
 'description': 'Manage GC Tech portfolio and work with global PMO team on cross territory project management. Manage Tech operation, including but not limited to annual AOP, monthly forecast, landing estimation, contract routing, payment delivery and incident management. Work with 3rd party to make sure the contingency of Asia Salesforce applications, RMS, CIM and etc. Take the role as Tech PM to support Asia finance and operations team on ad-hoc requirements.'}

In [22]:
type(json_res)

dict

In [26]:
import pandas as pd

df = pd.read_csv("my_portfolio_atlique.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [34]:
job = json_res
job['skills']

['PMP Certificate',
 'Microsoft Office (Word, Excel, PowerPoint, Visio, Project)',
 'Jira',
 'Confluence',
 'Smartsheet',
 'Fluent in both written and verbal Chinese and English']

In [36]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [38]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Pradhum Niroula, a business development executive at Niroula Group. Niroula Group is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Niroula Group 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Niroula Group portfolio: {link_list}
        Remember you are Pradhum Niroula, BDE at Niroula Group. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Tech OPS Solutions for Seamless Business Operations

Dear Hiring Manager,

I came across the job description for a Sr. Software Engineer - Tech OPS role at your esteemed organization, and I am excited to introduce Niroula Group as a potential partner to fulfill your technical requirements. With our expertise in AI and software consulting, we can help streamline your business operations and enhance overall efficiency.

Our team has extensive experience in managing complex projects, including portfolio management, agile project management, and tech operations. We are well-versed in utilizing tools like Jira, Confluence, Smartsheet, and Microsoft Office to ensure seamless project execution. Our expertise in managing annual AOP, monthly forecast, landing estimation, contract routing, payment delivery, and incident management can help your organization achieve its goals.

As a seasoned business development executive at Niroula Group, I would like to highlight our capabilitie